# 파이단틱 사용하기 

파이단틱은 입력된 데이터의 유효성과 형식을 검증하고 특정 데이터 형식으로 명확하게 표현할 때 사용하는 라이브러리 



In [5]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    # Base 모델은 ticker가 문자열인지, period가 문자열인지, 필수 값이 비었는지 검증
    # ...은 반드시 있어야 하는 값 
    # description은 필드에 대한 설명
    # title은 필드의 제목
    ticker : str = Field(..., title = '주식 코드', description="주식 코드 ( 예 : AAPL )")
    period: str  = Field(..., title = '기간', description="주식 데이터 조회 기간( 예 : 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max )")

🪄 실제 사용 예시
data = StockHistoryInput(ticker="AAPL", period="1y")
print(data)


그러면 이런 객체가 생성됨:

ticker='AAPL' period='1y'


반대로 이런 입력이면:

StockHistoryInput(ticker=123, period=1)


➡️ 자동으로 에러 발생
(타입이 str이 아니라서)

In [3]:
!pip install yfinance

done
  Created wheel for peewee: filename=peewee-3.18.3-cp312-cp312-linux_x86_64.whl size=1016249 sha256=9561885f68e941bfc011627d9bc88b44be808ee81fb07831fc54fe8851ab90c5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e2/48/b6/675a31c56e50b8b343e1ffbb1d9209f0d95025e2cfa0bbeeed
Successfully built multitasking peewee
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [yfinance]8/9 [yfinance]soup4]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [26]:
import yfinance as yf
import pytz
from datetime import datetime
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
import dotenv
import json


dotenv.load_dotenv()


@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """
    주식 종목의 각각각 데이터를 조회하는 함수 

    """
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown()
    return history_md


@tool
def get_current_time(timezone: str, location: str) -> str:
    """현재 시각을 반환하는 함수
    Args:
        timezone (str): 시간대 ( 예: 'Asia/Seoul' ). 실제 존재하는 시간대여야함.
        location (str): 지역명. 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_time = f"{timezone} , {location}의 현재 시각은 {now} 입니다."
    print(location_and_time)
    return location_and_time

llm = ChatOpenAI(model="gpt-4o")

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}  

llm_with_tools = llm.bind_tools(tools)

In [ ]:
# 회사의 주가 변화에 관한 질문을 추가하고, llm_with_tools.invoke를 사용하여 답변을 요청
messages = []
messages.append(HumanMessage(content="테슬라는 한달 전에 비해 주가가 올랐어? 내렸어?"))

# chain 설정
parser = StrOutputParser()
chain = llm_with_tools | parser
chain.invoke(messages)

# 체인을 설정하지 않고, ai의 response 출력 결과 보기
response = llm_with_tools.invoke(messages)
print(response.tool_calls)
# [{'name': 'get_yf_stock_history', 
# 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 
# 'id': 'call_lAs1CtuqjTeePDIHt4FtM2t6', 'type': 'tool_call'}]

messages.append(response)

# messages 안에는 이미 아래와 같이 들어가 있으므로, ai가 어떠한 주체가 답변을 했는지 명확히 알 수 있다.
# {
#   "type": "ai",
#   "content": "안녕하세요! 무엇을 도와드릴까요?"
# }

[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_lAs1CtuqjTeePDIHt4FtM2t6', 'type': 'tool_call'}]


In [ ]:
# gpt가 의도한 대로 함수를 실행하고 결과를 메시지로 추가할 수 있도록 다음과 같이 코드를 만듬
# 위에 출력된 결과를 보면 reponse안의 tool_calls에 리스트 형태로 실행할 함수들이 들어 있는 형태였다.
# 따라서 reponse.tool_calls로 for문을 이용해 gpt가 계획했던 함수를 모두 실행하고 그 결과들을 messages에 추가


for tool_call in response.tool_calls:
    # tool_dict[tool_call["name"]] 부분은 tool 객체인 
    # name='get_yf_stock_history' 
    # description='주식 종목의 각각각 데이터를 조회하는 함수' 
    # args_schema=<class 'langchain_core.utils.pydantic.get_yf_stock_history'> 
    # func=<function get_yf_stock_history at 0x...>

    selected_tool = tool_dict[tool_call["name"]]
    print(selected_tool)
    # print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    # print(tool_msg)

name='get_yf_stock_history' description='주식 종목의 각각각 데이터를 조회하는 함수' args_schema=<class 'langchain_core.utils.pydantic.get_yf_stock_history'> func=<function get_yf_stock_history at 0x7fc451352340>


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [46]:
# 데코레이터

# 1초 뒤, 문자열 출력
import time 
def timer(func):
    def wrapper(*args):
        time.sleep(3)
        return func(*args)
    return wrapper
@timer
def print_text(text):
    print(text)

print_text("Hello, World!")

Hello, World!


## @tool 데코레이터가 하는 일

원래는 이 함수가 그냥 파이썬 함수였습니다.

def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    ...


여기에 @tool을 붙이면:

@tool
def get_yf_stock_history(...):
    ...


LangChain이 이 함수를 감싸서, 내부적으로 다음과 같은 정보를 가진 Tool 객체로 바꿔 줍니다.

name : "get_yf_stock_history"

description : docstring

args_schema : Pydantic 기반 입력 스키마

func : 실제 파이썬 함수

.invoke(...) : 이 도구를 호출하는 메서드